In [11]:
import pandas as pd
from pathlib import Path

data_dir = Path(".//data")

In [12]:
hdb_props = pd.read_csv(data_dir/"hdb-property-information.csv")

In [13]:
def format_address(street, block):
    return f"{block} {street}, Singapore"


In [14]:
hdb_props['address'] = hdb_props.apply(lambda x: format_address(x.street, x.blk_no), axis=1)

print(hdb_props['address'][0:10])

0             1 BEACH RD, Singapore
1      1 BEDOK STH AVE 1, Singapore
2        1 CANTONMENT RD, Singapore
3         1 CHAI CHEE RD, Singapore
4    1 CHANGI VILLAGE RD, Singapore
5            1 DELTA AVE, Singapore
6             1 DOVER RD, Singapore
7           1 EUNOS CRES, Singapore
8           1 EVERTON PK, Singapore
9        1 GEYLANG SERAI, Singapore
Name: address, dtype: object


In [15]:
access_key = "dd5a3b4533f25374ec5e263c1a165a91"

In [16]:
import requests
from functools import cache

endpoint = "http://api.positionstack.com/v1/forward"

@cache
def get_geolocation(address):
    params = {
        "access_key": access_key,
        "query": address,
        "limit": 3,
    }
    
    response = requests.get(endpoint, params=params)
    
    if response.status_code != 200:
        print(f"Positionstack API returned {response.status_code} status code: {response.text}")
        return
    
    json_response = response.json()
    entry = json_response['data'][0]
    
    lat = entry['latitude']
    long = entry['longitude']
    confidence = entry['confidence']
    _type = entry['type']
    postal_code = entry['postal_code']
    trusted = confidence >= 1 and _type == "address"
    
    if not trusted:
        print(f"Maybe bad result (conf.: {confidence}, type: {_type}, code: {postal_code})")
    
    return [lat, long, postal_code, confidence, _type]

In [19]:
#Test
get_geolocation(format_address("CHOA CHU KANG CRES", "690"))

[1.402664, 103.752028, None, 1, 'address']

In [21]:
def load_geolocations():
    chunk_size = 500
    start_offset = 10500

    total_rows = len(hdb_props)
    i = 0


    # Loop over each chunk of rows
    for start in range(start_offset, total_rows, chunk_size):
        end = start + chunk_size
        results = []

        chunk = hdb_props.iloc[start:end]

        # Loop over each row in current chunk
        for index, row in chunk.iterrows():
            block = row["blk_no"]
            street = row["street"]
            address = row["address"]

            print(f'[{i}] Retrieving geolocation for {address}')

            [lat, long, postal_code, confidence, _type] = get_geolocation(address)

            results.append({
                "block": block, 
                "street_name": street, 
                "latitude": lat, 
                "longitude": long,
                "postal_code": postal_code,
                "confidence": confidence, 
                "type": _type,
            })

            i = i+1

        chunk_df = pd.DataFrame(results)
        chunk_df.to_csv(data_dir / f"address_geolocations_{start}_{end}.csv", index=False)

    # Create DataFrame for the remaining rows (if any)
    if results:
        result_df = pd.DataFrame(results)
        result_df.to_csv(data_dir / f"address_geolocations_{end}_{total_rows}.csv", index=False)

# only uncomment if you really want to run it. Will take approx. 3h
# load_geolocations()

In [36]:
import re
import os

pattern = r"address_geolocations_\d+_\d+.csv"
files = [filename for filename in os.listdir(data_dir) if re.match(pattern, filename) ]

dfs = []
for file in files:
    df = pd.read_csv(data_dir / file)
    dfs.append(df)

all_geolocations = pd.concat(dfs)

all_geolocations.to_csv(data_dir / "address_geolocations.csv", index=False)

In [37]:
total_rows = all_geolocations.shape[0]
print("Total number of rows:", total_rows)

low_confidence_rows = all_geolocations[all_geolocations["confidence"] < 1].shape[0]
print("Number of rows where 'confidence' is less than 1:", low_confidence_rows)

type_not_address_rows = all_geolocations[all_geolocations["type"] != "address"].shape[0]
print("Number of rows where 'type' is not 'address':", type_not_address_rows)

Total number of rows: 12848
Number of rows where 'confidence' is less than 1: 1682
Number of rows where 'type' is not 'address': 1682


In [51]:
resale_prices = pd.read_csv(data_dir / "resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")

hdb_props_geolocation = pd.merge(resale_prices, all_geolocations, how="left", on=["block", "street_name"])

print(f"Total sales rows: {hdb_props_geolocation.shape[0]}")
low_confidence_rows = hdb_props_geolocation[hdb_props_geolocation["confidence"] < 1]
print(f"Total low confidence rows: {low_confidence_rows.shape[0]}")
print(f"Difference: {hdb_props_geolocation.shape[0] - low_confidence_rows.shape[0]}")

Total sales rows: 154293
Total low confidence rows: 15782
Difference: 138511


In [53]:
hdb_props_geolocation.sample(10)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,latitude,longitude,postal_code,confidence,type
69495,2020-03,JURONG WEST,5 ROOM,204,BOON LAY DR,04 TO 06,120.0,Standard,1976,55 years 04 months,360000.0,1.344385,103.715143,640204.0,1.0,address
53940,2019-07,JURONG WEST,3 ROOM,213,BOON LAY PL,07 TO 09,66.0,Improved,1974,54 years 03 months,215000.0,1.349400,103.712868,640213.0,1.0,address
63376,2019-12,BUKIT BATOK,3 ROOM,409,BT BATOK WEST AVE 4,07 TO 09,68.0,New Generation,1988,67 years 05 months,233000.0,1.316382,103.833828,NaN,0.6,locality
33318,2018-08,BEDOK,4 ROOM,529,BEDOK NTH ST 3,10 TO 12,92.0,New Generation,1979,60 years 01 month,390000.0,1.334266,103.926552,460529.0,1.0,address
133890,2022-08,BEDOK,4 ROOM,34,BEDOK STH AVE 2,10 TO 12,90.0,Improved,1977,54 years 01 month,432888.0,1.322014,103.938350,460034.0,1.0,address
108121,2021-09,SEMBAWANG,5 ROOM,483,ADMIRALTY LINK,10 TO 12,111.0,Improved,2004,81 years 08 months,480000.0,1.455998,103.815416,NaN,1.0,address
116792,2021-04,YISHUN,4 ROOM,840,YISHUN ST 81,10 TO 12,103.0,Model A,1988,66 years 01 month,463000.0,1.415610,103.833620,NaN,1.0,address
82395,2020-10,JURONG WEST,5 ROOM,673C,JURONG WEST ST 65,04 TO 06,110.0,Improved,2002,81 years 03 months,415000.0,1.344743,103.701318,643673.0,1.0,address
135493,2022-08,TAMPINES,5 ROOM,488A,TAMPINES AVE 9,07 TO 09,121.0,Improved,1989,65 years 09 months,650000.0,1.360200,103.956327,520488.0,1.0,address
42483,2018-12,WOODLANDS,5 ROOM,688B,WOODLANDS DR 75,04 TO 06,110.0,Premium Apartment,2004,84 years 03 months,370000.0,1.443809,103.807946,NaN,1.0,address


array([1. , 0.6])